Setting up modules

In [ ]:
import ee
import geemap
import geopandas as gpd
import requests
from shapely import geometry
import pandas as pd
from io import StringIO

ee.Authenticate()
# ee.Initialize()
ee.Initialize(project="ee-fasoriano")


Choosing the area of interest (AOI)

In [ ]:
# URL to the GeoJSON file on GitHub
geojson_url = 'https://raw.githubusercontent.com/fralasor/autocam-workshop/main/rsws_region_bboxes.geojson'

# Fetch the GeoJSON file
geojson_data = requests.get(geojson_url).json()

# Create a map instance
map1 = geemap.Map()

# Load GeoJSON into a GeoDataFrame
gdf = gpd.GeoDataFrame.from_features(geojson_data['features'])
# display(gdf)

# Select the AOI based on its ID
# Bago City for this workshop
feature = gdf[gdf['id'] == 0].squeeze()

# Convert the selected feature geometry to a Shapely object
geom = feature.geometry

# Convert Shapely geometry to GeoJSON
geom_geojson = geometry.mapping(geom)
aoi_bbox = ee.FeatureCollection(ee.Geometry(geom_geojson))

# AOI Visualization
aoi_style = {
    'color': 'red', 
    'width': 2,
    'lineType': 'solid',
    'fillColor': '00000000',
}

# Add the selected AOI to the map
map1.addLayer(aoi_bbox.style(**aoi_style), {}, "Selected AOI")
map1.centerObject(aoi_bbox)

# Display the map
map1


Parameters

In [ ]:
"""
You can also use the two following lines to set your AOI based on a shp/json file  
# extent_path = fr"D:/fsori/Documents/AUTOCAM/Study Area/tagum_boundbox.geojson"
# AOI = geemap.geojson_to_ee(extent_path)
"""

AOI = aoi_bbox
START_DATE = '2023-01-01'
END_DATE = '2023-03-31'
CLOUD_FILTER = 60
CLD_PRB_THRESH = 40
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST_km = 2
BUFFER_m = 100
output_name = 'bago_2023_q1_s2'

Cloud masking using S2 cloud probability dataset (s2cloudless) dataset

In [ ]:
# Defining functions for cloud-masking
def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))
    
def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST_km input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST_km*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER_m*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)

def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)

# Sentinel 2 Visualization
s2_visualization = {
    'min': 0,
    'max': 10000,
    'bands': ['B4', 'B3', 'B2'],
    'gamma': 1.2
}


Calling S2 image collection & Adding cloud-free composites to basemap

In [ ]:
# Fetching Sentinel 2 image collection within the set time period
s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filterBounds(AOI)
        .filterDate(START_DATE, END_DATE)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER))
        )

# APPLYING S2 CLOUD MASK
s2_sr_cld_col = get_s2_sr_cld_col(AOI, START_DATE, END_DATE)
s2_sr_median = (s2_sr_cld_col.map(add_cld_shdw_mask)
                             .map(apply_cld_shdw_mask)
                             .median())

# Add median cloud-free composite
map1.addLayer(s2_sr_median.clip(aoi_bbox), s2_visualization, name='s2 median cloudless')

# Bring AOI polygon to top
map1.remove(aoi_bbox)
map1.addLayer(aoi_bbox.style(**aoi_style), {}, "Selected AOI")

# Display map
map1

In [ ]:
print('All band names:', s2_sr_median.bandNames().getInfo())

Loading training and testing data

Setting visualization parameters

In [ ]:
train_data_path = "https://raw.githubusercontent.com/fralasor/autocam-workshop/main/bago_train_2023.geojson"
test_data_path = "https://raw.githubusercontent.com/fralasor/autocam-workshop/main/bago_test_2023.geojson"

# Fetch the GeoJSON file
train_data = requests.get(train_data_path).json()
test_data = requests.get(test_data_path).json()

lc_labels_dict = {
    1:"inland water",
    2:"grassland",
    3:"fallow land",
    4:"crops",
    5:"coastal water",
    6:"built-up",
    7:"barren",
    8:"trees",
    9:"brush",
    10:"fishpond",
    11:"mangrove"
    }

lc_colors = [
    '#1f77b4',  # inland water
    '#2ca02c',  # grassland
    '#ff7f0e',  # fallow land
    '#d62728',  # crops
    '#17becf',  # coastal water
    '#9467bd',  # built-up
    '#8c564b',  # barren
    '#e377c2',  # trees
    '#7f7f7f',  # brush
    '#bcbd22',  # fishpond
    '#ffbb78'   # mangrove
]

rf_vis_param = {'min': 1, 'max': 11, 'palette': lc_colors}

Choosing and Training Supervised Classification Algorithm

In [ ]:
bands = s2_sr_median.bandNames().getInfo()
label = "lc_code"

training = s2_sr_median.select(bands).sampleRegions(**{
    'collection':train_data,
    'properties':[label],
    'scale':10,
})

classifier = ee.Classifier.smileRandomForest(100).train(training, label, bands) # .train()

result = s2_sr_median.select(bands).classify(classifier) #.classify() 'trained' => trained classifier



Displaying Land Cover Map

In [ ]:
map1.addLayer(result.clip(AOI), rf_vis_param, 'classified')
map1.add_legend(title="Land Cover", labels=list(lc_labels_dict.values()), colors=lc_colors)
map1

Accuracy Assessment

In [ ]:
#Validation

validation = s2_sr_median.select(bands).sampleRegions(**{
    'collection':test_data,
    'properties':[label],
    'scale':10,
})

test = validation.classify(classifier)

confusionMatrix = test.errorMatrix('lc_code','classification')

# print('RF Consumer\'s accuracy (Omission)', confusionMatrix.consumersAccuracy().getInfo())
# print('RF Producer\'s accuracy (Commission)', confusionMatrix.producersAccuracy().getInfo())

In [ ]:
print('RF Overall accuracy', confusionMatrix.accuracy().getInfo())
print('RF Kappa Coefficient', confusionMatrix.kappa().getInfo())

# Create dataframe for confusion matrix
conf_df = pd.DataFrame(confusionMatrix.getInfo())
conf_df.drop(0, axis=1, inplace=True)
conf_df.drop(0, axis=0, inplace=True)

# Display confusion matrix
conf_df.columns = list(lc_labels_dict.values())
conf_df['index'] = list(lc_labels_dict.values())
conf_df.set_index('index', inplace=True)
display(conf_df)

Exporting land cover map to Google Drive

In [ ]:
task = ee.batch.Export.image.toDrive(**{
    'image': result.clip(aoi_bbox),
    'description': output_name+'_classified',
    'folder': 'autocam_s2',
    'scale': 10,
    'region': aoi_bbox.geometry(),
    'crs': 'EPSG:4326'
    
})

task.start()
print(f"Running task {task.status()['task_type']} of {task.status()['description']}")

while(task.status()['state'] != 'COMPLETED'):
    continue

task.status()


Global Human Settlement Layer (GHSL) Building Characteristics

In [ ]:
map2 = geemap.Map()
map2.add_basemap('SATELLITE')

ghsl = ee.ImageCollection("JRC/GHSL/P2023A/GHS_BUILT_C").select('built_characteristics')
ghsl_class_table = """
Value	Color	Description
1	#718c6c	open spaces, low vegetation surfaces
2	#8ad86b	open spaces, medium vegetation surfaces
3	#c1ffa1	open spaces, high vegetation surfaces
4	#01b7ff	open spaces, water surfaces
5	#ffd501	open spaces, road surfaces
11	#d28200	built spaces, residential, building height <= 3m
12	#fe5900	built spaces, residential, 3m < building height <= 6m
13	#ff0101	built spaces, residential, 6m < building height <= 15m
14	#ce001b	built spaces, residential, 15m < building height <= 30m
15	#7a000a	built spaces, residential, building height > 30m
21	#ff9ff4	built spaces, non-residential, building height <= 3m
22	#ff67e4	built spaces, non-residential, 3m < building height <= 6m
23	#f701ff	built spaces, non-residential, 6m < building height <= 15m
24	#a601ff	built spaces, non-residential, 15m < building height <= 30m
25	#6e00fe	built spaces, non-residential, building height > 30m
"""

ghsl_class_table_df = pd.read_csv(StringIO(ghsl_class_table), sep="\t")
display(ghsl_class_table_df)

ghsl_vis_param = {'min': 1, 'max': 25, 'palette': list(ghsl_class_table_df['Color'])}

legend_dict = geemap.legend_from_ee(ghsl_class_table)
map2.add_legend(title="GHSL 2018", legend_dict=legend_dict)
map2.addLayer(ghsl, ghsl_vis_param, 'GHSL')
# Add the selected AOI to the map
map2.addLayer(aoi_bbox.style(**aoi_style), {}, "Selected AOI")
map2.centerObject(aoi_bbox)

In [ ]:
map2

Dynamic World Land Cover dataset

In [ ]:
# Construct a collection of corresponding Dynamic World and Sentinel-2 for
# inspection. Filter the DW and S2 collections by region and date.
START = ee.Date('2023-03-01')
END = START.advance(30, 'day')

col_filter = ee.Filter.And(
    ee.Filter.bounds(AOI),
    ee.Filter.date(START, END),
)

dw_col = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filter(col_filter)
s2_col = ee.ImageCollection('COPERNICUS/S2_HARMONIZED').filter(col_filter)

# Join corresponding DW and S2 images (by system:index).
dw_s2_col = ee.Join.saveFirst('s2_img').apply(
    dw_col,
    s2_col,
    ee.Filter.equals(leftField='system:index', rightField='system:index'),
)

# Extract an example DW image and its source S2 image.
dw_image = ee.Image(dw_s2_col.first())
s2_image = ee.Image(dw_image.get('s2_img'))

# Create a visualization that blends DW class label with probability.
# Define list pairs of DW LULC label and color.
CLASS_NAMES = [
    'water',
    'trees',
    'grass',
    'flooded_vegetation',
    'crops',
    'shrub_and_scrub',
    'built',
    'bare',
    'snow_and_ice',
]

VIS_PALETTE = [
    '419bdf',
    '397d49',
    '88b053',
    '7a87c6',
    'e49635',
    'dfc35a',
    'c4281b',
    'a59b8f',
    'b39fe1',
]

# Create an RGB image of the label (most likely class) on [0, 1].
dw_rgb = (
    dw_image.select('label')
    .visualize(min=0, max=8, palette=VIS_PALETTE)
    .divide(255)
)

# Get the most likely class probability.
top1_prob = dw_image.select(CLASS_NAMES).reduce(ee.Reducer.max())

# Create a hillshade of the most likely class probability on [0, 1]
top1_prob_hillshade = ee.Terrain.hillshade(top1_prob.multiply(100)).divide(255)

# Combine the RGB image with the hillshade.
dw_rgb_hillshade = dw_rgb.multiply(top1_prob_hillshade)

# Display the Dynamic World visualization with the source Sentinel-2 image.
map3 = geemap.Map()
map3.centerObject(AOI)
map3.add_layer(
    s2_image,
    {'min': 0, 'max': 3000, 'bands': ['B4', 'B3', 'B2']},
    'Sentinel-2 L1C',
)
map3.add_layer(
    dw_rgb_hillshade,
    {'min': 0, 'max': 0.65},
    'Dynamic World V1 - label hillshade',
)
map3.add_legend(title="Dynamic World LC", labels=CLASS_NAMES, colors=VIS_PALETTE)
map3